In [220]:
import pandas as pd
from pathlib import Path
import torch
import transformers
import random
import transformers
from nltk import pos_tag
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/antoniomenta/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [221]:
RESOURCES_DIR = Path("/Users/antoniomenta/PycharmProjects/simpleTextCLEF/resources")
DATASETS_PATH = RESOURCES_DIR / "datasets"
WORD_EMBEDDINGS_NAME = "glove.42B.300d"
DUMPS_DIR = RESOURCES_DIR / "DUMPS"

In [222]:
SIMPLETEXT_DATASET_PATH = DATASETS_PATH / "simpleText/task 3"
SIMPLETEXT_TRAIN_PATH = SIMPLETEXT_DATASET_PATH / "train"

In [223]:
complex_text = pd.read_csv(SIMPLETEXT_TRAIN_PATH / "simpleText.train.complex.txt", header=None, sep="\t", names=["original_text"])
simple_text = pd.read_csv(SIMPLETEXT_TRAIN_PATH / "simpleText.train.simple.txt", header=None, sep="\t",names=["simple_text"])

In [224]:
sentences_pairs = pd.concat([complex_text, simple_text], axis=1)
sentences_pairs

,original_text,simple_text
0,"In the modern era of automation and robotics, ...",Current academic and industrial research is in...
1,With the ever increasing number of unmanned ae...,Drones are increasingly used in the civilian a...
2,Due to guidelines set by the governments regar...,Governments set guidelines on the operation ce...
3,In an attempt to achieve the above mentioned t...,Researchers propose data-driven solutions allo...
4,Derived from the classic image classification ...,"The algorithm, based on the Inception model, d..."
...,...,...
643,Bodybuilders generally train with moderate loa...,Bodybuilders train with moderate loads and sho...
644,"Powerlifters, on the other hand, routinely tra...","Powerlifters, on the other hand, train with hi..."
645,Although both groups are known to display impr...,Although both groups are known to display impr...
646,It has been shown that many factors mediate th...,It has been shown that many factors mediate th...


In [225]:
model = transformers.AutoModelWithLMHead.from_pretrained('lordtt13/COVID-SciBERT')
tokenizer = transformers.AutoTokenizer.from_pretrained('lordtt13/COVID-SciBERT')

/Users/antoniomenta/PycharmProjects/simpleTextCLEF/venv/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:742: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [226]:
def mask_sentence_prediction(text, topk=10):
    sentence_tokens = [token[0] for token in tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)]
    #print(pos_tag(sentence_tokens))
    pos_tagging_sentence = pos_tag(sentence_tokens)
    #print(len(pos_tagging_sentence))
    candidates = 0
    predictions = []
    for i, (word, pos) in enumerate(pos_tagging_sentence):
        #print(f"i:{i}, word:{word}, pos:{pos}")
        if pos in ['NNS', 'NN', 'VBP', 'RB', 'VBG','VBD']:
            candidates +=1
            sentence_masked = sentence_tokens.copy()
            sentence_masked[i] = "[MASK]"
            sentence_final = " ".join(sentence_masked)
            #print(sentence_final)
            inputs = tokenizer(sentence_final, return_tensors="pt")
            token_logits = model(**inputs).logits
            mask_token_index = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]
            mask_token_logits = token_logits[0,mask_token_index,:]
            top_n_tokens = torch.topk(mask_token_logits,topk,dim=1).indices[0].tolist()
            predicted_tokens = [tokenizer.decode([token]) for token in top_n_tokens]
            if word in predicted_tokens:
                predictions.append(predicted_tokens.index(word))


    predictions = [np.log(pred+1) for pred in predictions]
    #print(predictions)
    if candidates == 0:
            return np.log(1 + tokenizer.vocab_size)
    return np.quantile(predictions, 0.75)

def mask_sentence_prediction(text, topk=10):
    sentence_tokens = [token[0] for token in tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)]
    #print(pos_tag(sentence_tokens))
    pos_tagging_sentence = pos_tag(sentence_tokens)
    #print(len(pos_tagging_sentence))
    candidates = 0
    predictions = 0
    for i, (word, pos) in enumerate(pos_tagging_sentence):
        #print(f"i:{i}, word:{word}, pos:{pos}")
        if pos in ['NNS', 'NN', 'VBP', 'RB', 'VBG','VBD']:
            candidates +=1
            sentence_masked = sentence_tokens.copy()
            sentence_masked[i] = "[MASK]"
            sentence_final = " ".join(sentence_masked)
            #print(sentence_final)
            inputs = tokenizer(sentence_final, return_tensors="pt")
            token_logits = model(**inputs).logits
            mask_token_index = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]
            mask_token_logits = token_logits[0,mask_token_index,:]
            top_n_tokens = torch.topk(mask_token_logits,topk,dim=1).indices[0].tolist()
            predicted_tokens = [tokenizer.decode([token]) for token in top_n_tokens]
            if word in predicted_tokens:
                predictions +=1

    if candidates != 0:
        return predictions/candidates
    else:
        return 1

In [227]:
#mask_sentence_prediction("The computational efficiency of MAVNet enables the drone to fly at high speeds of up to 6m/sec.")

In [228]:
def check_pair_sentence(complex, simple):
    complex_feature = mask_sentence_prediction(complex)
    simple_feature = mask_sentence_prediction(simple)
    #print(f"Complex feature: {complex_feature}. Simple feature: {simple_feature}")
    if simple_feature !=0:
        return simple_feature/complex_feature
    else:
        return 1

In [229]:
complex = "The computational efficiency of MAVNet enables the drone to fly at high speeds of up to 6m/sec."
simple = "MAVNet computational efficiency enables the drone to fly up to 6m/sec."

In [230]:
check_pair_sentence(complex, simple)

0.3667257913420846

In [231]:
results = []
for i,row in sentences_pairs.iterrows():
    value = check_pair_sentence(row['original_text'], row['simple_text'])
    results.append(value)
    if i % 5 == 0:
        print(i)

0
5


/var/folders/3_/9_kdtx5d2d782tr9kwff27zh0000gn/T/ipykernel_6203/2748310643.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  return simple_feature/complex_feature


10
15
20
25
30
35
40


IndexError: cannot do a non-empty take from an empty axes.

In [232]:
results

[1,
 0.2789429456511298,
 0.4,
 1.4678924870096008,
 0.8221267860233528,
 0.9693609377704336,
 0.1934264036172708,
 0.6020599913279624,
 inf,
 0.3667257913420846,
 0.7958852231401551,
 1.0,
 1,
 0.375,
 inf,
 1.9387218755408673,
 inf,
 1.695677926806633,
 inf,
 1.1509775004326936,
 3.113283334294875,
 0.946394630357186,
 1.5849625007211563,
 1.8787894123356965,
 1.9182958340544898,
 1.0,
 1.5777262166065846,
 1,
 1,
 1.1973573641278246,
 1.3634165139304208,
 0.7924812503605781,
 0.7859128611337362,
 1.0,
 0.3509193652572005,
 1.5849625007211563,
 0.6309297535714574,
 2.2924812503605785,
 1,
 1.8905606060552682,
 1.4387218755408673,
 1.7924812503605783,
 1.0654648767857287,
 0.946394630357186]

In [233]:
len(results)

44

In [ ]:
fig = plt.figure(figsize=(10,6))
sns.distplot(results,label="LM")
fig.legend(labels=['LM'])
plt.show()

In [ ]:
import numpy as np

In [ ]:
np.mean(results)

In [ ]:
tokenizer.vocab_size